In [46]:
import numpy as np
import cv2
from itertools import cycle
import pickle
import pathlib
import math
import tqdm
import scipy.io
from matplotlib import pyplot as plt
import scipy.io
import h5py
import re
from lxml import etree as ET
import scipy.signal as sig
import pandas as pd
from scipy.stats import kde
from BlockSync_current import BlockSync
import UtilityFunctions_newOE as uf
from scipy import signal
import bokeh
import seaborn as sns

In [47]:
def create_distance_plot(distances, top_dist_to_show=500):
    # Create cumulative distribution plot
    sns.set(style="whitegrid")
    fig, axs = plt.subplots(2, figsize=(6, 6), dpi=150)
    
    axs[0].set_title('Cumulative Euclidean Distances for Camera Jitter', fontsize=15)
    axs[0].set_ylabel('Cumulative \n % of Frames')
    axs[0].set_xlim(0, top_dist_to_show)
    axs[0].grid(False)
    
    # Create histogram and cumulative distribution
    sns.kdeplot(distances, cumulative=True, label='Left Eye', ax=axs[0], linewidth=4, c='black')
    
    axs[1].hist(distances, bins=np.linspace(0, top_dist_to_show, 20), log=False, color='black')

    # Set title and labels
    title = 'Image displacement histogram'
    axs[1].set_title(title, fontsize=15)
    axs[1].set_xlabel('Euclidean Displacement [$\mu$m]', fontsize=15)
    axs[1].set_xscale('linear')
    axs[1].set_yscale('linear')
    axs[1].set_ylabel('Frame count', fontsize=15)

    # Adjust tick label sizes
    axs[1].tick_params(axis='x', which='major', labelsize=15)

    # Set white background and black text
    axs[1].set_facecolor('white')
    axs[1].title.set_color('black')
    axs[1].xaxis.label.set_color('black')
    axs[1].yaxis.label.set_color('black')
    axs[1].tick_params(colors='black')
    axs[1].grid(False)

    plt.tight_layout()

    return fig, axs
def add_intermediate_elements(input_vector, gap_to_bridge):
    # Step 1: Calculate differences between each element
    differences = np.diff(input_vector)

    # Step 2: Add intervening elements based on the diff_threshold
    output_vector = [input_vector[0]]
    for i, diff in enumerate(differences):
        if diff < gap_to_bridge:
            # Add intervening elements
            output_vector.extend(range(input_vector[i] + 1, input_vector[i + 1]))

        # Add the next element from the original vector
        output_vector.append(input_vector[i + 1])

    return np.sort(np.unique(output_vector))

def find_jittery_frames(block, eye, max_distance, diff_threshold, gap_to_bridge=6):
    
    #input checks
    if eye not in ['left', 'right']:
        print(f'eye can only be left/right, your input: {eye}')
        return None
    # eye setup
    if eye == 'left':
        jitter_dict = block.le_jitter_dict
        eye_frame_col = 'L_eye_frame'
    elif eye == 'right':
        jitter_dict = block.re_jitter_dict
        eye_frame_col = 'R_eye_frame'
    
    df_dict = {'left':block.le_df,
               'right':block.re_df}
    
    df = pd.DataFrame.from_dict(jitter_dict)
    indices_of_highest_drift = df.query("top_correlation_dist > @max_distance").index.values
    diff_vec = np.diff(df['top_correlation_dist'].values)
    diff_peaks_indices = np.where(diff_vec > diff_threshold)[0]
    video_indices = np.concatenate((diff_peaks_indices, indices_of_highest_drift))
    print(f'the diff based jitter frame exclusion gives: {np.shape(diff_peaks_indices)}')
    print(f'the threshold based jitter frame exclusion gives: {np.shape(indices_of_highest_drift)}')
    
    # creates a bridged version of the overly jittery frames (to contend with single frame outliers)
    video_indices = add_intermediate_elements(video_indices, gap_to_bridge=gap_to_bridge)
    # This is the input you should give to the BlockSync.remove_eye_datapoints function (which already maps it to the df) 
    
    
    # translates the video indices to le/re dataframe rows
    df_indices_to_remove = df_dict[eye].loc[df_dict[eye][eye_frame_col].isin(video_indices)].index.values
    
    return df_indices_to_remove, video_indices

def bokeh_plotter(data_list, label_list,
                  plot_name='default',
                  x_axis='X', y_axis='Y',
                  peaks=None, peaks_list=False, export_path=False):
    """Generates an interactive Bokeh plot for the given data vector.
    Args:
        data_list (list or array): The data to be plotted.
        label_list (list of str): The labels of the data vectors
        plot_name (str, optional): The title of the plot. Defaults to 'default'.
        x_axis (str, optional): The label for the x-axis. Defaults to 'X'.
        y_axis (str, optional): The label for the y-axis. Defaults to 'Y'.
        peaks (list or array, optional): Indices of peaks to highlight on the plot. Defaults to None.
        export_path (False or str): when set to str, will output the resulting html fig
    """
    color_cycle = cycle(bokeh.palettes.Category10_10)
    fig = bokeh.plotting.figure(title=f'bokeh explorer: {plot_name}',
                                x_axis_label=x_axis,
                                y_axis_label=y_axis,
                                plot_width=1500,
                                plot_height=700)

    for i, vec in enumerate(range(len(data_list))):
        color = next(color_cycle)
        data_vector = data_list[vec]
        if label_list is None:
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"Line {len(fig.renderers)}")
        elif len(label_list) == len(data_list):
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"{label_list[i]}")
        if peaks is not None and peaks_list is True:
            fig.circle(peaks[i], data_vector[peaks[i]], size=10, color=color)

    if peaks is not None and peaks_list is False:
        fig.circle(peaks, data_vector[peaks], size=10, color='red')

    if export_path is not False:
        print(f'exporting to {export_path}')
        bokeh.io.output.output_file(filename=str(export_path / f'{plot_name}.html'), title=f'{plot_name}')
    bokeh.plotting.show(fig)
    
def play_video_with_ellipses_rotation(block, eye, path_to_video=False, xflip=False, transformation_matrix=None, phi_in_radians=False):
    if eye == 'left':
        video_path = block.le_videos[0]
        ellipse_dataframe = block.left_eye_data
    elif eye == 'right':
        video_path = block.re_videos[0]
        ellipse_dataframe = block.right_eye_data
    else:
        raise ValueError(f"eye can only be 'left' or 'right'")
    
    if video_path is not False:
        video_path = path_to_video

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Loop through each frame
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
    
        if not ret:
            # Break the loop if the video is finished
            break
        
        # Optionally flip the frame along the x-axis
        if xflip:
            frame = cv2.flip(frame, 1)

        # Apply transformation matrix if provided
        if transformation_matrix is not None:
            frame = cv2.warpAffine(frame, transformation_matrix, (frame.shape[1], frame.shape[0]))

        # Get the corresponding ellipse data for the current frame
        current_frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        try:
            current_frame_data = ellipse_dataframe.iloc[ellipse_dataframe.query('eye_frame == @current_frame_num').index[0]]
        except IndexError:
            continue

        # Extract ellipse parameters
        if transformation_matrix is not None:
            try:
                center_x = int(current_frame_data['center_x'])
                center_y = int(current_frame_data['center_y'])
                width = int(current_frame_data['width'])
                height = int(current_frame_data['height'])
                if phi_in_radians:
                    phi = np.deg2rad(float(current_frame_data['phi']))
                else:
                    phi = float(current_frame_data['phi'])
                
                # Draw the ellipse on the frame
                cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
                
                # Add text to the frame
                text = f'ellipse angle: {phi}'
                cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
                
                # Display the frame
                cv2.imshow('Video with Ellipses', frame)
            
                # Check for the 'q' key to quit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            except ValueError:
                continue
        else:
            try:
                center_x = int(current_frame_data['center_x'])
                center_y = int(current_frame_data['center_y'])
                width = int(current_frame_data['width'])
                height = int(current_frame_data['height'])
                if phi_in_radians:
                    phi = np.deg2rad(float(current_frame_data['phi']))
                else:
                    phi = float(current_frame_data['phi'])
        
                # Draw the ellipse on the frame
                cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
                
                # Add text to the frame
                text = f'ellipse angle: {phi}'
                cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                
                # Display the frame
                cv2.imshow('Video with Ellipses', frame)
            
                # Check for the 'q' key to quit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            except ValueError:
                continue

    # Release video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()
    
def play_video_with_ellipses_rotation_plus_major_axis(block, eye, path_to_video=False, xflip=False, transformation_matrix=None):
    if eye == 'left':
        video_path = block.le_videos[0]
        ellipse_dataframe = block.left_eye_data
    elif eye == 'right':
        video_path = block.re_videos[0]
        ellipse_dataframe = block.right_eye_data
    else:
        raise ValueError(f"eye can only be 'left' or 'right'")
    
    if video_path is not False:
        video_path = path_to_video

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Loop through each frame
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
    
        if not ret:
            # Break the loop if the video is finished
            break
        
        # Optionally flip the frame along the x-axis
        if xflip:
            frame = cv2.flip(frame, 1)

        # Apply transformation matrix if provided
        if transformation_matrix is not None:
            frame = cv2.warpAffine(frame, transformation_matrix, (frame.shape[1], frame.shape[0]))

        # Get the corresponding ellipse data for the current frame
        current_frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        try:
            current_frame_data = ellipse_dataframe.iloc[ellipse_dataframe.query('eye_frame == @current_frame_num').index[0]]
        except IndexError:
            continue

        # Extract ellipse parameters
        try:
            center_x = int(current_frame_data['center_x'])
            center_y = int(current_frame_data['center_y'])
            width = int(current_frame_data['major_ax'])
            height = int(current_frame_data['minor_ax'])
            phi = np.deg2rad(float(current_frame_data['phi']))  # Convert angle to radians
            
            # Draw the ellipse on the frame
            cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
            
            # Calculate endpoints of major axis
            axis_length = max(width, height) / 2
            sin_phi = np.sin(phi)
            cos_phi = np.cos(phi)
            x1 = int(center_x + axis_length * cos_phi)
            y1 = int(center_y + axis_length * sin_phi)
            x2 = int(center_x - axis_length * cos_phi)
            y2 = int(center_y - axis_length * sin_phi)
            
            # Draw major axis
            cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            
            # Add text to the frame
            text = f'ellipse angle: {np.rad2deg(phi)}'
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Display the frame
            cv2.imshow('Video with Ellipses', frame)
        
            # Check for the 'q' key to quit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        except ValueError:
            continue

    # Release video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()
    
def get_frame_count(video_path):
        """
        Get the number of frames for the video in the specified path using OpenCV.
    
        Parameters:
            video_path (str): Path to the video file.
    
        Returns:
            int: Number of frames in the video.
        """
        
        # Open the video file
        cap = cv2.VideoCapture(video_path)
    
        # Check if the video file is opened successfully
        if not cap.isOpened():
            print("Error: Could not open the video file.")
            return -1
    
        # Get the total number of frames in the video
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
        # Release the VideoCapture object
        cap.release()
    
        return frame_count


# Block Instantiation

In [132]:

# define a single block to synchronize and finally export l/r_eye_data csv files:
# this step creates block_collection - a list of BlockSync objects of interest
block_numbers = [15]
bad_blocks = [6] # 
experiment_path = pathlib.Path(r"Z:\Nimrod\experiments")
animal = 'PV_126'

block_collection = uf.block_generator(block_numbers=block_numbers,
                                      experiment_path=experiment_path,
                                      animal=animal,
                                      bad_blocks=bad_blocks,regev=True)
# create a block_dict object for ease of access:
block_dict = {}
for b in block_collection:
    block_dict[str(b.block_num)] = b
block = block_collection[0]

instantiated block number 015 at Path: Z:\Nimrod\experiments\PV_126\2024_08_13\block_015, new OE version
Found the sample rate for block 015 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 015
got it!


In [133]:
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import pathlib
from tqdm import tqdm

def synchronize_block_for_non_60fps_acquisition(self, export=True, overwrite=False, target_frame_rate=60, margin_of_error=0.1):
    """
    Synchronize the video frames to a target frame rate using interpolation.
    """
    # Check if previously exported file exists
    if pathlib.Path(self.analysis_path / 'blocksync_df.csv').exists() and not overwrite:
        self.blocksync_df = pd.read_csv(pathlib.Path(self.analysis_path / 'blocksync_df.csv'), engine='python')
        print('blocksync_df loaded from analysis folder')
        return self.blocksync_df
    
    print('Creating blocksync_df')

    # Define start and end times
    start_time = max([self.arena_vid_first_t, self.r_vid_first_t, self.l_vid_first_t])
    end_time = min([self.arena_vid_last_t, self.r_vid_last_t, self.l_vid_last_t])

    # Extract TTLs and calculate frame rate
    arena_ttls = self.oe_events.query('@start_time < Arena_TTL < @end_time')['Arena_TTL'].values
    arena_frame_rate = self.sample_rate / np.median(np.diff(arena_ttls))

    if not (target_frame_rate - margin_of_error <= arena_frame_rate <= target_frame_rate + margin_of_error):
        print(f"Arena video frame rate is {arena_frame_rate:.2f} Hz. Adjusting to {target_frame_rate} FPS.")

        # Define target time base
        original_time = np.cumsum(np.insert(np.diff(arena_ttls), 0, 0)) / self.sample_rate
        target_time = np.arange(0, original_time[-1], 1 / target_frame_rate)

        # Interpolate using linear method
        interpolator = interp1d(original_time, arena_ttls, kind='linear', fill_value='extrapolate')
        new_arena_ttl = interpolator(target_time).astype(int)
    else:
        print(f"Arena video frame rate is {arena_frame_rate:.2f} Hz, within acceptable range. No adjustment needed.")
        new_arena_ttl = arena_ttls

    # Create a synchronization DataFrame
    arena_tf = self.oe_events.query('@start_time < Arena_TTL < @end_time')[['Arena_TTL', 'Arena_TTL_frame']]
    r_eye_tf = self.oe_events.query('@start_time < Arena_TTL < @end_time or Arena_TTL != Arena_TTL')[['R_eye_TTL', 'R_eye_TTL_frame']].dropna()
    l_eye_tf = self.oe_events.query('@start_time < Arena_TTL < @end_time or Arena_TTL != Arena_TTL')[['L_eye_TTL', 'L_eye_TTL_frame']].dropna()

    # Build the synchronization DataFrame
    self.blocksync_df = pd.DataFrame(columns=['Arena_frame', 'L_eye_frame', 'R_eye_frame'], index=new_arena_ttl)

    for t in tqdm(new_arena_ttl, desc='Synchronizing Frames'):
        arena_frame = arena_tf['Arena_TTL_frame'].iloc[self.get_closest_frame(t, arena_tf['Arena_TTL'])]
        l_eye_frame = l_eye_tf['L_eye_TTL_frame'].iloc[self.get_closest_frame(t, l_eye_tf['L_eye_TTL'])]
        r_eye_frame = r_eye_tf['R_eye_TTL_frame'].iloc[self.get_closest_frame(t, r_eye_tf['R_eye_TTL'])]
        self.blocksync_df.loc[t] = [arena_frame, l_eye_frame, r_eye_frame]

    print('Created blocksync_df')
    if export:
        self.blocksync_df.to_csv(self.analysis_path / 'blocksync_df.csv')
        print(f'Exported blocksync_df to {self.analysis_path}/blocksync_df.csv')

    return self.blocksync_df


In [134]:
# This step is used to quickly go over the analyzed blocks and load their internal data
for block in block_collection:
    block.handle_arena_files()
    block.parse_open_ephys_events()
    block.handle_eye_videos()
    block.get_eye_brightness_vectors()
    #synchronize_block_for_non_60fps_acquisition(block,export=True,overwrite=True,target_frame_rate=60,margin_of_error=0.1)
    #block.synchronize_block()
    block.synchronize_block_for_non_60fps_acquisition(export=True,
                                                      overwrite=True,
                                                      target_frame_rate=60,
                                                      margin_of_error=0.5)
    block.create_eye_brightness_df(threshold_value=20)
    
    # if the code fails here, go to manual synchronization
    block.import_manual_sync_df()

handling arena files
Arena video Names:
front_20240813T150642.mp4
left_20240813T150642.mp4
right_20240813T150642.mp4
top_20240813T150642.mp4
running parse_open_ephys_events...
block 015 has a parsed events file, reading...
handling eye video files
converting videos...
converting files: ['Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_015\\eye_videos\\LE\\126_w10_640x480_60hz_experiment_1_recording_0\\126_w10.h264', 'Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_015\\eye_videos\\RE\\126_w10_640x480_60hz_experiment_1_recording_0\\126_w10.h264'] 
 avoiding conversion on files: ['Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_015\\eye_videos\\LE\\126_w10_640x480_60hz_experiment_1_recording_0\\126_w10_LE.mp4', 'Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_015\\eye_videos\\RE\\126_w10_640x480_60hz_experiment_1_recording_0\\126_w10.mp4']
The file Z:\Nimrod\experiments\PV_126\2024_08_13\block_015\eye_videos\RE\126_w10_640x480_60hz_experiment_1_recording_0\126_w10.mp4 already

100%|██████████| 97999/97999 [00:34<00:00, 2842.70it/s]


created blocksync_df
exported blocksync_df to Z:\Nimrod\experiments\PV_126\2024_08_13\block_015\analysis/ blocksync_df.csv
creating Z:\Nimrod\experiments\PV_126\2024_08_13\block_015\analysis/eye_brightness_df.csv
there is no manual sync file for block 015, manually sync the block


In [142]:
for block in block_collection:
    block.read_dlc_data(overwrite=True, export=True)
    block.calibrate_pixel_size(10)

c:\users\marks3\pycharmprojects\pythonproject3\venv\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████| 97844/97844 [00:59<00:00, 1640.92it/s]



 ellipses calculation complete


100%|██████████| 97609/97609 [00:57<00:00, 1683.10it/s]
C:\Users\MarkS3\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\reshape\merge.py:1112: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():



 ellipses calculation complete
created le / re dataframes
exporting to analysis folder
got the calibration values from the analysis folder


In [143]:
for block in block_collection:
    block.get_jitter_reports(export=True, overwrite=False, remove_led_blinks=False, sort_on_loading=True)

Computing Cross-Correlation:   3%|▎         | 2513/97611 [00:10<06:13, 254.85frame/s]C:\Users\MarkS3\PycharmProjects\pythonProject3\BlockSync_current.py:1826: RuntimeWarning: invalid value encountered in divide
  out = out / np.sqrt(image * template)
Computing Cross-Correlation:  96%|█████████▌| 93564/97846 [05:38<00:15, 279.80frame/s]C:\Users\MarkS3\PycharmProjects\pythonProject3\BlockSync_current.py:1826: RuntimeWarning: divide by zero encountered in divide
  out = out / np.sqrt(image * template)
Computing Cross-Correlation: 100%|█████████▉| 97845/97846 [05:54<00:00, 276.38frame/s]


results saved to Z:\Nimrod\experiments\PV_126\2024_08_13\block_015\analysis\jitter_report_dict.pkl
Got the jitter report - check out re/le_jitter_dict attributes


In [144]:
for block in block_collection:
#    block.get_jitter_reports(export=True, overwrite=True, remove_led_blinks=False, sort_on_loading=True)
    block.correct_jitter()
    block.find_led_blink_frames(plot=True)
    block.remove_led_blinks_from_eye_df(export=True)

The right eye std of the X coord was 23.860579658160653
After correction it is: 19.824572626903237
The right eye std of the Y coord was 56.85015255377097
After correction it is: 77.64178916691681

 The left eye std of the X coord was 24.01711128529211
After correction it is: 24.361775099773702

 The left eye std of the Y coord was 14.691032176124786
After correction it is: 16.310097849348423


100%|██████████| 65/65 [00:00<00:00, 6483.77it/s]

collecting left-eye data
data length is 97846
z_score length is 97846



100%|██████████| 65/65 [00:00<00:00, 8791.96it/s]

collecting right eye data
data length is 97611
z_score length is 97611


removed led blink data from le / re dataframes
exported nan filled dataframes to csv


In [148]:
df_inds_to_remove_l, vid_inds_l = find_jittery_frames(block,'left',max_distance=20, diff_threshold=5, gap_to_bridge=24)
df_inds_to_remove_r, vid_inds_r = find_jittery_frames(block,'right',max_distance=20, diff_threshold=5, gap_to_bridge=24)

# These are verification plots for the jitter outlier removal functions:
# to verify, I want a bokeh explorable:
rdf = pd.DataFrame.from_dict(block.re_jitter_dict)
ldf = pd.DataFrame.from_dict(block.le_jitter_dict)

the diff based jitter frame exclusion gives: (256,)
the threshold based jitter frame exclusion gives: (409,)
the diff based jitter frame exclusion gives: (98,)
the threshold based jitter frame exclusion gives: (60487,)


In [149]:
bokeh_plotter([rdf.top_correlation_dist], ['drift_distance'], peaks=vid_inds_r)

In [150]:
bokeh_plotter([ldf.top_correlation_dist], ['drift_distance'], peaks=vid_inds_l)

In [151]:
block.remove_eye_datapoints_based_on_video_frames('right', indices_to_nan=vid_inds_r)
block.remove_eye_datapoints_based_on_video_frames('left', indices_to_nan=vid_inds_l)

removed 60564 from the right eye dataframe
removed 1238 from the left eye dataframe


# Data rotation

In [152]:
block.rotate_data_according_to_frame_ref('left')

Please select two points on the frame.
left rotation matrix: 
 [[  0.95197877  -0.30616403  88.84615877]
 [  0.30616403   0.95197877 -86.44739498]] 
 left rotation angle: 
 -17.828208332058207
left data rotated


In [153]:
block.rotate_data_according_to_frame_ref('right')

Please select two points on the frame.
right rotation matrix: 
 [[  0.95312427   0.30257913 -57.61875727]
 [ -0.30257913   0.95312427 108.07549826]] 
 right rotation angle: 
 17.61257784292383
right data rotated


In [154]:
block.create_eye_data()

Index(['Arena_TTL', 'R_eye_frame', 'ms_axis', 'center_x_rotated',
       'center_y_rotated', 'phi_rotated', 'width', 'height', 'major_ax',
       'minor_ax', 'ratio'],
      dtype='object')
   OE_timestamp  eye_frame   ms_axis    center_x    center_y       phi  \
0      676740.0        0.0  33837.00  362.731420  169.259462  5.894529   
1      677073.0        2.0  33853.65  362.868702  169.244679  5.662257   
2      677397.0        3.0  33869.85  362.861714  169.181252  5.298216   
3      677730.0        4.0  33886.50  362.725648  169.216733  5.331653   
4      678053.0        5.0  33902.65  362.811217  169.333283  4.814047   

       width     height   major_ax   minor_ax     ratio  
0  45.637936  37.335848  45.637936  37.335848  1.222362  
1  45.585475  37.188742  45.585475  37.188742  1.225787  
2  45.640489  37.209324  45.640489  37.209324  1.226587  
3  45.716606  37.162466  45.716606  37.162466  1.230182  
4  45.532021  37.073076  45.532021  37.073076  1.228170  
Index(['Arena_TTL

C:\Users\MarkS3\PycharmProjects\pythonProject3\BlockSync_current.py:2778: RuntimeWarning: All-NaN axis encountered
  df['major_ax'] = np.nanmax(df[['width', 'height']], axis=1)
C:\Users\MarkS3\PycharmProjects\pythonProject3\BlockSync_current.py:2779: RuntimeWarning: All-NaN axis encountered
  df['minor_ax'] = np.nanmin(df[['width', 'height']], axis=1)


In [157]:
# This bit examine the ellipses to verify phi jump issues
block.right_eye_data['phi'] = block.right_eye_data['phi'] + 90

In [89]:
block.left_eye_data['phi'] = block.left_eye_data['phi'] - 90

# Rotation eye data Verification

In [158]:
# right eye inspection after rotation
#path_to_video = [x for x in pathlib.Path(block.re_videos[0]).parent.iterdir() if '.mp4' in str(x.name) and 'DLC' in str(x.name)][0]
path_to_video = [x for x in pathlib.Path(block.re_videos[0]).parent.iterdir() if '.mp4' in str(x.name)][0]
print(path_to_video)
play_video_with_ellipses_rotation(block=block ,eye='right', path_to_video=str(path_to_video) ,xflip=True, transformation_matrix=block.right_rotation_matrix)

Z:\Nimrod\experiments\PV_126\2024_08_13\block_015\eye_videos\RE\126_w10_640x480_60hz_experiment_1_recording_0\126_w10.mp4


In [163]:
# left eye inspection
#path_to_video = [x for x in pathlib.Path(block.le_videos[0]).parent.iterdir() if '.mp4' in str(x.name) and 'DLC' in str(x.name)][0]
path_to_video = [x for x in pathlib.Path(block.le_videos[0]).parent.iterdir() if '.mp4' in str(x.name)][0]
play_video_with_ellipses_rotation(block=block,eye='left', path_to_video=str(path_to_video) ,xflip=True, transformation_matrix=block.left_rotation_matrix)

# Eye videos relative lag correction

In [164]:
# block integrated version:
block.find_led_blink_frames()
l_df, r_df = block.correct_relative_eye_drift_based_on_LED_lights_out(verification_plots=False)
block.left_eye_data = l_df
block.right_eye_data = r_df

100%|██████████| 65/65 [00:00<00:00, 8150.12it/s]

collecting left-eye data
data length is 97846
z_score length is 97846
collecting right eye data
data length is 97611
z_score length is 97611
missing frame at 21
missing frame at 24
missing frame at 26
97999 97999
0
0
0 S
1 S
2 S
3 S
4 S
5 S
6 S
7 S
8 S
9 S
10 S
11 S
12 S
13 S
14 S
15 S
16 S
17 S
18 S
19 S
20 S
21 S
22 S
23 S


In [165]:
l_df, r_df = block.correct_eye_sync_based_on_OE_LED_events()

double row at 1
double row at 3
missing frame at 21
missing frame at 24
missing frame at 26
The correction employed was -2.0, 
check the output and overwirte the left/right eye data dfs when happy, then re-export


In [94]:
block.left_eye_data = l_df
block.right_eye_data = r_df

In [44]:
# I get weird behavior from my OE_LED sync - here is where I'm testing what's really up
# helper functrions:
def find_middle_values(series):
    # Create a DataFrame from the series
    data = pd.DataFrame({'index': series.index, 'value': series.values})

    # Identify chunks of consecutive indices
    data['chunk'] = (data['index'].diff() != 1).cumsum()

    # Function to get the middle value of each chunk
    def get_middle_value(chunk):
        # Calculate the position of the middle value (0-based index)
        middle_idx = (len(chunk) - 1) // 2
        return chunk.iloc[middle_idx]['value']

    # Group by chunk and apply the function to find the middle value
    middle_values = data.groupby('chunk').apply(get_middle_value).tolist()

    return middle_values

oe_led_blinks = block.oe_events[['LED_driver']].query('LED_driver == LED_driver').values
ms_timestamps = oe_led_blinks.T / 20
ms_axis = block.left_eye_data.ms_axis.values
ms_blink_frames = []
# The timestamps now correspond with the real time axis and not the down-sampled arena frames time markers -
# the following code corrects that and finds the closest frames
for t in ms_timestamps[0]:
    ms_blink_frames.append(ms_axis[np.argmin(np.abs(ms_axis - t))])

ms_blink_times = np.array(ms_blink_frames)
le_frame_blinks = block.left_eye_data[block.left_eye_data.eye_frame.isin(block.led_blink_frames_l)].ms_axis
le_blinks_ms = find_middle_values(le_frame_blinks)
re_frame_blinks = block.right_eye_data[block.right_eye_data.eye_frame.isin(block.led_blink_frames_r)].ms_axis
re_blinks_ms = find_middle_values(re_frame_blinks)


# End of Synchronization Pipeline

# Data export steps:

In [166]:
def export_eye_data_2d_w_rotation_matrix(block):
    """
    This function saves the eye dataframes to two csv files
    :param block: The current blocksync class with verifiec re/le dfs
    :return: None
    """
    block.right_eye_data.to_csv(block.analysis_path / 'right_eye_data.csv')
    block.left_eye_data.to_csv(block.analysis_path / 'left_eye_data.csv')
    rotation_dict = {'left_rotation_matrix': block.left_rotation_matrix,
                'left_rotation_angle':  block.left_rotation_angle,
                'right_rotation_matrix':block.right_rotation_matrix,
                'right_rotation_angle': block.right_rotation_angle}
    with open(block.analysis_path / 'rotate_eye_data_params.pkl', 'wb') as file:
        pickle.dump(rotation_dict, file)
        print(f'eye dataframes and rotation matrix saved to: {file}')
        
export_eye_data_2d_w_rotation_matrix(block)

eye dataframes and rotation matrix saved to: <_io.BufferedWriter name='Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_015\\analysis\\rotate_eye_data_params.pkl'>


In [37]:
block.final_sync_df

,Arena_TTL,Arena_frame,L_eye_frame,R_eye_frame,L_values,R_values
0,857974.0,392.0,136.0,69.0,-2.313608,-1.473610
1,858307.0,393.0,137.0,70.0,-2.307578,-1.472873
2,858630.0,393.0,138.0,71.0,-2.307772,-1.466903
3,858963.0,394.0,139.0,72.0,-2.311223,-1.458486
4,859287.0,394.0,140.0,73.0,-2.311346,-1.461533
...,...,...,...,...,...,...
154064,51439069.0,77424.0,NaN,NaN,NaN,NaN
154065,51439402.0,77425.0,NaN,NaN,NaN,NaN
154066,51439725.0,77425.0,NaN,NaN,NaN,NaN
154067,51440058.0,77426.0,NaN,NaN,NaN,NaN


# Manual Synchronization steps - only use when needed and after block instantiation

In [135]:
block.get_eyes_diff_list(2)

index error on position 59 out of 60
index error on position 91 out of 92
The suspected lag between eye cameras is -5.0 with the direction ['right', 'early']


In [136]:
block.fix_eye_synchronization()

created manual_sync_df attribute for the block


In [137]:
block.blocksync_df

,Arena_frame,L_eye_frame,R_eye_frame
Arena_TTL,,,
676740.0,497,5,0
677073.0,498,6,2
677397.0,498,7,3
677730.0,499,8,4
678053.0,499,9,5
...,...,...,...
32834371.0,49494,96856,96654
32834704.0,49495,96857,96655
32835027.0,49495,96858,96656


In [139]:
# use this to manually shift L\R eye synchronization
eye_to_move = 'R'
block.move_eye_sync_manual(cols_to_move=[f'{eye_to_move}_eye_frame',f'{eye_to_move}_values'],step=-5)

In [140]:
block.full_sync_verification(ms_axis=False,with_arena=False)

In [141]:
block.export_manual_sync_df()
block.import_manual_sync_df()

In [111]:
def full_sync_verification(self):
    from bokeh.plotting import figure, show, curdoc
    from bokeh.models import Slider, ColumnDataSource, CustomJS
    from bokeh.layouts import column
    import numpy as np
    data_dict = {
    'L_values': [None if np.isnan(x) or np.isinf(x) else x for x in
    self.manual_sync_df['L_values']],
    'R_values': [None if np.isnan(x) or np.isinf(x) else x for x in
    self.manual_sync_df['R_values']]
    }
    source = ColumnDataSource(data=dict(
    x_axis=self.manual_sync_df.index / self.sample_rate,
    left_y=[None if np.isnan(x) or np.isinf(x) else x for x in
    self.manual_sync_df['L_values'].shift(0)],
    right_y=[None if np.isnan(x) or np.isinf(x) else x for x in
    self.manual_sync_df['R_values'].shift(0)]
    ))
    bokeh_fig = figure(
    title=f'Full Synchronization Verification',
    x_axis_label='Seconds',
    y_axis_label='Brightness Z_Score',
    plot_width=1500,
    plot_height=700
    )
    left_line = bokeh_fig.line('x_axis', 'left_y', source=source,
    legend_label='Left_eye_values', line_width=1,
    line_color='blue')
    right_line = bokeh_fig.line('x_axis', 'right_y', source=source,
    legend_label='Right_eye_values', line_width=1,
    line_color='red')
    slider_left = Slider(start=-500, end=500, value=0, step=1, title="Left Eye Shift")
    slider_right = Slider(start=-500, end=500, value=0, step=1, title="Right Eye Shift")
    callback = CustomJS(args=dict(source=source, slider_left=slider_left,
    slider_right=slider_right, data_dict=data_dict),
    code="""
    const plot_data = source.data;
    const left_shift = slider_left.value;
    const right_shift = slider_right.value;
    const left_values = data_dict['L_values'];
    const right_values = data_dict['R_values'];
    const length = plot_data['x_axis'].length;
    for (let i = 0; i < length; i++) {
    plot_data['left_y'][i] = (i + left_shift >= 0 && i +
    left_shift < length) ? left_values[i + left_shift] : null;
    plot_data['right_y'][i] = (i + right_shift >= 0 && i +
    right_shift < length) ? right_values[i + right_shift] : null;
    }
    console.log("Left shift: ", left_shift, "Right shift: ",
    right_shift); // Debug output
    source.change.emit();
    """)
    slider_left.js_on_change('value', callback)
    slider_right.js_on_change('value', callback)
    layout = column(bokeh_fig, slider_left, slider_right)
    curdoc().add_root(layout)
    show(layout)


In [138]:
full_sync_verification(block)

In [38]:
block.manual_sync_df.rename(columns={'Unnamed: 0.1': 'Arena_TTL'},inplace=True)

In [39]:
block.manual_sync_df

,Arena_TTL,Arena_frame,L_eye_frame,R_eye_frame,L_values,R_values
0,913655,845.0,6.0,2.0,-0.970385,-0.877140
1,913988,846.0,6.0,3.0,-0.970385,-0.872642
2,914321,846.0,8.0,4.0,-0.969849,-0.871553
3,914655,847.0,9.0,5.0,-0.966254,-0.868974
4,914988,847.0,10.0,6.0,-0.970299,-0.867236
...,...,...,...,...,...,...
121570,41436988,62567.0,121799.0,121794.0,-0.277042,0.023131
121571,41437321,62567.0,121799.0,NaN,-0.277042,NaN
121572,41437655,62568.0,121800.0,NaN,-0.285377,NaN
121573,41437988,62568.0,121802.0,NaN,-0.269196,NaN


In [167]:
block.left_eye_data

,OE_timestamp,eye_frame,ms_axis,center_x,center_y,phi,width,height,major_ax,minor_ax,ratio
0,676740.0,5.0,33837.00,319.518120,177.992945,-15.443761,46.028471,40.458124,46.028471,40.458124,1.137682
1,677073.0,6.0,33853.65,319.537921,177.984564,-15.536813,46.187965,40.480872,46.187965,40.480872,1.140982
2,677397.0,7.0,33869.85,319.510882,178.011487,-15.462979,46.079456,40.440812,46.079456,40.440812,1.139430
3,677730.0,8.0,33886.50,319.491438,178.003452,-14.895568,45.985501,40.541131,45.985501,40.541131,1.134293
4,678053.0,9.0,33902.65,319.491147,178.091727,-14.170065,45.875067,40.475895,45.875067,40.475895,1.133392
...,...,...,...,...,...,...,...,...,...,...,...
97994,32834371.0,96856.0,1641718.55,340.761068,109.354315,-24.174997,41.280409,28.235806,41.280409,28.235806,1.461988
97995,32834704.0,96857.0,1641735.20,340.931360,109.326126,-24.160139,41.655405,28.352723,41.655405,28.352723,1.469185
97996,32835027.0,96858.0,1641751.35,340.753870,109.435444,-24.996913,42.075840,28.446548,42.075840,28.446548,1.479119
97997,32835360.0,96859.0,1641768.00,341.181379,109.345112,-23.876929,41.828722,28.359923,41.828722,28.359923,1.474924


In [ ]:
# This bit of code goes over blocks and collects the median distance between the rostral and caudal edges
import os
from ellipse import LsqEllipse
import scipy.stats as stats

def eye_tracking_analysis(dlc_video_analysis_csv, uncertainty_thr):
    """
    :param dlc_video_analysis_csv: the csv output of a dlc analysis of one video, already read by pandas with header=1
    :param uncertainty_thr: The confidence P value to use as a threshold for datapoint validity in the analysis
    :returns ellipse_df: a DataFrame of ellipses parameters (center, width, height, phi, size) for each video frame

    """
    # import the dataframe and convert it to floats
    data = dlc_video_analysis_csv
    data = data.iloc[1:].apply(pd.to_numeric)

    # sort the pupil elements to dfs: x and y, with p as probability
    pupil_elements = np.array([x for x in data.columns if 'Pupil' in x])

    # get X coords
    pupil_xs_before_flip = data[pupil_elements[np.arange(0, len(pupil_elements), 3)]]

    # flip the data around the midpoint of the x-axis (shooting the eye through a camera flips right and left)
    pupil_xs = 320 * 2 - pupil_xs_before_flip

    # get Y coords (no need to flip as opencv conventions already start with origin at top left of frame
    # and so, positive Y is maintained as up in a flipped image as we have)
    pupil_ys = data[pupil_elements[np.arange(1, len(pupil_elements), 3)]]
    pupil_ps = data[pupil_elements[np.arange(2, len(pupil_elements), 3)]]

    # rename dataframes for masking with p values of bad points:
    pupil_ps = pupil_ps.rename(columns=dict(zip(pupil_ps.columns, pupil_xs.columns)))
    pupil_ys = pupil_ys.rename(columns=dict(zip(pupil_ys.columns, pupil_xs.columns)))
    good_points = pupil_ps > uncertainty_thr
    pupil_xs = pupil_xs[good_points]
    pupil_ys = pupil_ys[good_points]

    # Do the same for the edges
    edge_elements = np.array([x for x in data.columns if 'edge' in x])
    edge_xs_before_flip = data[edge_elements[np.arange(0, len(edge_elements), 3)]]
    edge_xs = 320*2 - edge_xs_before_flip
    edge_ys = data[edge_elements[np.arange(1, len(edge_elements), 3)]]
    edge_ps = data[edge_elements[np.arange(2,len(edge_elements),3)]]
    edge_ps = edge_ps.rename(columns=dict(zip(edge_ps.columns,edge_xs.columns)))
    edge_ys = edge_ys.rename(columns=dict(zip(edge_ys.columns,edge_xs.columns)))
    good_edge_points = edge_ps < uncertainty_thr
    
    # work row by row to figure out the ellipses
    ellipses = []
    caudal_edge_ls = []
    rostral_edge_ls = []
    for row in tqdm.tqdm(range(1, len(data) - 1)):
        # first, take all the values, and concatenate them into an X array
        x_values = pupil_xs.loc[row].values
        y_values = pupil_ys.loc[row].values
        X = np.c_[x_values, y_values]

        # now, remove nan values, and check if there are enough points to make the ellipse
        X = X[~ np.isnan(X).any(axis=1)]

        # if there are enough rows for a fit, make an ellipse
        if X.shape[0] > 5:
            el = LsqEllipse().fit(X)
            center, width, height, phi = el.as_parameters()
            center_x = center[0]
            center_y = center[1]
            ellipses.append([center_x, center_y, width, height, phi])
        else:
            ellipses.append([np.nan, np.nan, np.nan, np.nan, np.nan])

        caudal_edge = [
            float(data['Caudal_edge'][row]),
            float(data['Caudal_edge.1'][row])
        ]
        rostral_edge = [
            float(data['Rostral_edge'][row]),
            float(data['Rostral_edge.1'][row])
        ]
        caudal_edge_ls.append(caudal_edge)
        rostral_edge_ls.append(rostral_edge)


    # ellipse_df = pd.DataFrame(columns=['center_x', 'center_y', 'width', 'height', 'phi'], data=ellipses)
    # a = np.array(ellipse_df['height'][:])
    # b = np.array(ellipse_df['width'][:])
    # ellipse_size_per_frame = a * b * math.pi
    # ellipse_df['ellipse_size'] = ellipse_size_per_frame
    ellipse_df = pd.Dataframe({
        'rostral_edge':rostral_edge_ls,
        'caudal_edge': caudal_edge_ls
    })
    

    print(f'\n ellipses calculation complete')
    
    ellipse_df[['caudal_edge_x', 'caudal_edge_y']] = pd.DataFrame(ellipse_df['caudal_edge'].tolist(), index=ellipse_df.index)
    ellipse_df[['rostral_edge_x', 'rostral_edge_y']] = pd.DataFrame(ellipse_df['rostral_edge'].tolist(), index=ellipse_df.index)
    
    return ellipse_df

def get_pixel_distance(df):
    distances = np.sqrt((df['caudal_edge_x'] - df['rostral_edge_x'])**2 + 
                        (df['caudal_edge_y'] - df['rostral_edge_y'])**2)
    
    mean_distance = np.nanmean(distances)
    std_distance = np.nanstd(distances)
    # Shapiro-Wilk Test
    shapiro_test = stats.shapiro(distances)
    print(f"Shapiro-Wilk Test: Statistic={shapiro_test.statistic}, p-value={shapiro_test.pvalue}")
    
    # Kolmogorov-Smirnov Test
    ks_test = stats.kstest(distances, 'norm', args=(mean_distance, std_distance))
    print(f"Kolmogorov-Smirnov Test: Statistic={ks_test.statistic}, p-value={ks_test.pvalue}")
    
    median_distance = np.median(distances)
    iqr_distance = stats.iqr(distances)
    print(f"Median Distance: {median_distance}")
    print(f"IQR: {iqr_distance}")
    print(f'mean = {mean_distance}')
    print(f'std = {std_distance}')
    return median_distance

R_pix_distance_dict = {}
L_pix_distance_dict = {}

for block in block_collection:
    print(f'working on {block}')
    pl = [i for i in os.listdir(block.r_e_path) if 'DLC' in i and '.csv' in i]
    if len(pl) > 1:
        pl = [i for i in pl if 'filtered' in i][0]
    else:
        pl = pl[0]
    R_csv  = pd.read_csv(block.r_e_path / pl, header=1)
    
    pl = [i for i in os.listdir(block.l_e_path) if 'DLC' in i and '.csv' in i]
    if len(pl) > 1:
        pl = [i for i in pl if 'filtered' in i][0]
    else:
        pl = pl[0]
    L_csv  = pd.read_csv(block.l_e_path / pl, header=1)
    R_ellipse_df = eye_tracking_analysis(R_csv,0.998)
    print('working on the right eye')
    R_pixel_distance = get_pixel_distance(R_ellipse_df)    
    L_ellipse_df = eye_tracking_analysis(L_csv,0.998)
    print('working on the left eye')
    L_pixel_distance = get_pixel_distance(L_ellipse_df)
    R_pix_distance_dict[block.block_num] = R_pixel_distance
    L_pix_distance_dict[block.block_num] = L_pixel_distance
    